# Laboratorio 5 - Parte 1

### Máquinas de Vectores de Soporte

### 2018-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co


## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer Integrante: Sidney Paola Aguirre Castro
#### Segundo Integrante: John Alexander Galeano Ospina

In [1]:
from __future__ import division
import numpy as np
import math
import matplotlib.pyplot as plt
#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")
%matplotlib inline

## Ejercicio 1

En este ejercicio deben resolver un problema de clasificación multi-clase usando una SVM. Teniendo en cuenta que la formulación original de la SVM sólo permite resolver problemas bi-clase, deben comparar los resultados obtenidos usando una estrategia **Uno vs Uno** con una estrategia **Uno vs El resto**.

Cargamos los datos:

In [6]:
from sklearn.datasets import load_digits
digits = load_digits
digits.data

AttributeError: 'function' object has no attribute 'data'

In [60]:
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
digits = load_digits(n_class=4)

#--------- preprocesamiento--------------------
pca = PCA(0.99, whiten=True)
data = pca.fit_transform(digits.data)

#---------- Datos a usar ----------------------
X = data
Y = digits.target

Consutar el manejo de la librería sklearn para entrenar un modelos SVM en: http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC. Para el caso multiclase la librería ya tiene implementada la estrategia **Uno vs Uno**, así que en este caso sólo deben llamar correctamente los métodos. Complete el siguiente código usando un clasificador basado en SVM:

In [47]:
import math
import numpy as np
from numpy import random
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier

#Validamos el modelo
Folds = 4
random.seed(19680801)
EficienciaTrain = np.zeros(Folds)
EficienciaVal = np.zeros(Folds)
skf = StratifiedKFold(n_splits=Folds)
psv_mean = np.zeros(4)
j = 0
for train, test in skf.split(X, Y):
    Xtrain = X[train,:]
    Ytrain = Y[train]
    Xtest = X[test,:]
    Ytest = Y[test]
    
    #Normalizamos los datos
    scaler = preprocessing.StandardScaler().fit(Xtrain)
    Xtrain = scaler.transform(Xtrain)
    Xtest = scaler.transform(Xtest)
    
    #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
    #Penalty parameter C of the error term.
    C = 1
    #Kernel ['linear',‘rbf’] Specifies the kernel type to be used in the algorithm.
    kernel = 'rbf'
    #Gamma 
    gamma = 0.01
    modelo = SVC(C, kernel , gamma)
    modelo.fit(Xtrain, Ytrain)
    
    #Validación
    Ytrain_pred = modelo.predict(Xtrain)
    Yest = modelo.predict(Xtest)
    
    #Evaluamos las predicciones del modelo con los datos de test
    EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
    EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
    
    psv = modelo.support_vectors_
    psv_mean[j] = psv.size/Xtrain.size
    
    
    j += 1
#psv = modelo.support_vectors_
#psv psv.size
print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
print(str(np.mean(psv_mean)))

Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.987506815990477+-0.002358145427521826
0.7402258510567252


Realice los experimientos necesarios para llenar la siguiente tabla:

In [34]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['lineal','lineal','lineal','lineal','lineal','lineal','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)
df_types["Eficiencia en validacion"][3] = "0.97077"
df_types["Intervalo de confianza"][3] = "0.01548"
df_types["% de Vectores de Soporte"][3] = "0.2620"
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

C:\Users\johng\Anaconda3\lib\site-packages\ipykernel\comm\comm.py:159: PerformanceWarning: indexing past lexsort depth may impact performance.
  self._msg_callback(msg)


Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [46]:
qgrid_widget.get_changed_df()

Eficiencia en validacion Intervalo de confianza  \
Kernel C       gamma                                                   
lineal 0.001   0.00                   0.31143                0.10080   
       0.010   0.00                   0.97069                0.02926   
       0.100   0.00                   0.98044                0.01661   
       1.000   0.00                   0.97077                0.01548   
       10.000  0.00                   0.97077                 0.0154   
       100.000 0.00                   0.97077                0.01548   
rbf    0.001   0.01                   0.31143                0.10080   
               0.10                   0.31143                0.10080   
               1.00                   0.31143                0,10080   
       0.010   0.01                   0.31143                0.10080   
               0.10                   0.31143                0.10090   
               1.00                   0.31143                0.10090   
       0.100   0.01                   0.78501                0.04249   
               0.10                   0.78501                0.04249   
               1.00                   0.78501                0.04249   
       1.000   0.01                   0.98750                0.00235   
               0.10                   0.98750                0.00235   
               1.00                   0.98750                0.00235   
       10.000  0.01                   0.98750                0.00235   
               0.10                   0.98750                0.00235   
               1.00                   0.98750                0.00235   
       100.000 0.01                   0.98750                0.00235   
               0.10                   0.98750                0.00235   
               1.00                   0.98750                0.00235   

                     % de Vectores de Soporte  
Kernel C       gamma                           
lineal 0.001   0.00                       1.0  
       0.010   0.00                    0.6083  
       0.100   0.00                    0.2865  
       1.000   0.00                    0.2620  
       10.000  0.00                    0.2620  
       100.000 0.00                    0.2620  
rbf    0.001   0.01                       1.0  
               0.10                       1.0  
               1.00                       1.0  
       0.010   0.01                       1.0  
               0.10                       1.0  
               1.00                       1.0  
       0.100   0.01                    0.9972  
               0.10                    0.9972  
               1.00                    0.9972  
       1.000   0.01                    0.7402  
               0.10                    0.7402  
               1.00                    0.7402  
       10.000  0.01                    0.7402  
               0.10                    0.7402  
               1.00                    0.7402  
       100.000 0.01                    0.7402  
               0.10                    0.7402  
               1.00                    0.7402

## Ejercicio 2

Cree dos funciones, una para entrenar un conjunto de modelos bajo la estrategia Uno vs el resto, usando como clasificador base una SVM. La segunda función debe usar el conjunto de modelos entrenados, y clasificar un conjunto de muestras de validación.

In [71]:
def TrainSVM_OnevsRest(C, kernel , gamma, n_classes, X, Y):
    model = []
    for i in range(n_classes):
        Yaux = np.copy(Y)
        Yaux[Y == i] = 1
        Yaux[Yaux != i] = 0
        clf = SVC(C, kernel , gamma, probability=True)
        clf.fit(X, Yaux)
        modelo.append(clf)
    return modelo
    

def ValidaSVM_OnevsRest(data, model):
    Ytrain_pred = model.predict(data)
    return Ytrain_pred

## Ejercicio 3

Use las funciones definidas en el punto anterior para llevar a cabo la prueba de simulación con el mismo conjunto de datos del punto 1.

In [72]:
import math
import numpy as np
from numpy import random
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier

Folds = 4
random.seed(19680801)
EficienciaTrain = np.zeros(Folds)
EficienciaVal = np.zeros(Folds)
skf = StratifiedKFold(n_splits=Folds)
psv_mean = np.zeros(4)
j = 0
for train, test in skf.split(X, Y):
    Xtrain = X[train,:]
    Ytrain = Y[train]
    Xtest = X[test,:]
    Ytest = Y[test]
    
    #Normalizamos los datos
    scaler = preprocessing.StandardScaler().fit(Xtrain)
    Xtrain = scaler.transform(Xtrain)
    Xtest = scaler.transform(Xtest)
    
    #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
    #Penalty parameter C of the error term.
    C = 1
    #Kernel ['linear',‘rbf’] Specifies the kernel type to be used in the algorithm.
    kernel = 'linear'
    #Gamma 
    gamma = 0
    modelo = TrainSVM_OnevsRest(C, kernel , gamma)
    #modelo.fit(Xtrain, Ytrain)    
    
    #Validación
    Ytrain_pred = ValidaSVM_OnevsRest(Xtrain, modelo)
    Yest = modelo.predict(Xtest)
    
    #psv = modelo.support_vectors_
    #psv_mean[j] = psv.size/Xtrain.size
    
    #Evaluamos las predicciones del modelo con los datos de test
    EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
    EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
    j += 1
        
print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
print(str(np.mean(psv_mean)))

AttributeError: 'OneVsRestClassifier' object has no attribute 'support_vectors_'

Realice los experimientos necesarios para llenar la siguiente tabla:

In [9]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['lineal','lineal','lineal','lineal','lineal','lineal','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)
df_types["Eficiencia en validacion"][3] = "0.97633"
df_types["Intervalo de confianza"][3] = "0.01837"
df_types["% de Vectores de Soporte"][3] = "0.2778"
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [40]:
qgrid_widget.get_changed_df()

Eficiencia en validacion Intervalo de confianza  \
Kernel C       gamma                                                   
lineal 0.001   0.00                   0.95393                0.03873   
       0.010   0.00                   0.96093                0.03470   
       0.100   0.00                   0.97352                0.02263   
       1.000   0.00                   0.97633                0.01837   
       10.000  0.00                   0.97633                0.01837   
       100.000 0.00                   0.97633                0.01837   
rbf    0.001   0.01                   0.97348                0.02403   
               0.10                   0.97348                0.02403   
               1.00                   0.97348                0.02403   
       0.010   0.01                   0.97351                0.02122   
               0.10                   0.97351                0.02122   
               1.00                   0.97351                0.02122   
       0.100   0.01                   0.97351                0.02122   
               0.10                   0.97351                0.02122   
               1.00                   0.97351                0.02122   
       1.000   0.01                   0.98887                0.00558   
               0.10                   0.98887                0.00558   
               1.00                   0.98887                0.00558   
       10.000  0.01                   0.99027                0.00458   
               0.10                   0.99027                0.00458   
               1.00                   0.99027                0.00458   
       100.000 0.01                   0.99027                0.00458   
               0.10                   0.99027                0.00458   
               1.00                   0.99027                0.00458   

                     % de Vectores de Soporte  
Kernel C       gamma                           
lineal 0.001   0.00                            
       0.010   0.00                            
       0.100   0.00                            
       1.000   0.00                    0.2778  
       10.000  0.00                            
       100.000 0.00                            
rbf    0.001   0.01                            
               0.10                            
               1.00                            
       0.010   0.01                            
               0.10                            
               1.00                            
       0.100   0.01                            
               0.10                            
               1.00                            
       1.000   0.01                            
               0.10                            
               1.00                            
       10.000  0.01                            
               0.10                            
               1.00                            
       100.000 0.01                            
               0.10                            
               1.00

*En las tablas el punto separa las cifras decimales, es decir 100.000 es 100 ó 1.000 es 1.